In [1]:
!pip install redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.9/268.9 kB 6.0 MB/s eta 0:00:0000:01


In [2]:
import redis

r = redis.Redis(host='redis', port=6379, decode_responses=True)

How many actors and movies are stored in Redis?

In [14]:
nb_actors = len([k for k in r.scan_iter("actor:*")])
nb_movies = len([k for k in r.scan_iter("movie:*")])

print(f"acteurs : {nb_actors}")
print(f"films : {nb_movies}")


acteurs : 1320
films : 922


List 5 actors born before 1980

In [15]:
results = []
for key in r.scan_iter("actor:*"):
    dob = r.hget(key, "date_of_birth")
    try:
        if int(dob) < 1980:
            first_name = r.hget(key, "first_name")
            last_name = r.hget(key, "last_name")
            results.append((first_name, last_name, dob))
    except:
        pass
for actor in results[:5]:
    print(actor)

('Bill', 'Nighy', '1949')
('Ralph', 'Fiennes', '1962')
('Kevin', 'Costner', '1955')
('Isabella', 'Rossellini', '1952')
('Cassi', 'Davis', '1964')


Retrieve the genre and rating of the movie "The Imitation Game".

In [16]:
for key in r.scan_iter("movie:*"):
    title = r.hget(key, "title")
    if title == "The Imitation Game":
        genre = r.hget(key, "genre")
        rating = r.hget(key, "rating")
        print(f"Genre : {genre} | Note : {rating}")
        break

Genre : Biography | Note : 8.5


List the top 5 highest-rated movies.

In [7]:
movies = []

for key in r.scan_iter("movie:*"):
    title = r.hget(key, "title")
    try:
        rating = float(r.hget(key, "rating"))
        movies.append((title, rating))
    except:
        pass

top5 = sorted(movies, key=lambda x: x[1], reverse=True)[:5]
for m in top5:
    print(m)

('Boy 9', 9.4)
('Vegas (doc)', 9.4)
('The Shawshank Redemption', 9.3)
('Ween Live in Chicago', 9.2)
('Over Canada: An Aerial Adventure', 9.1)


How many movies have a rating above 7.5?

In [17]:
count = 0
for key in r.scan_iter("movie:*"):
    try:
        rating = float(r.hget(key, "rating"))
        if rating > 7.5:
            count += 1
    except:
        pass

print(f"{count}")

183


Update the rating of the movie "The Imitation Game" to 8.5.

In [18]:
for key in r.scan_iter("movie:*"):
    if r.hget(key, "title") == "The Imitation Game":
        r.hset(key, "rating", 8.5)
        print("update to  8.5")
        break

update to  8.5


Add a new actor: "Zendaya", born in 1996.

In [19]:
new_id = 1
while r.exists(f"actor:{new_id}"):
    new_id += 1

r.hset(f"actor:{new_id}", mapping={
    "first_name": "Zendaya",
    "last_name": "",
    "date_of_birth": "1996"
})

print(f"Zendaya ajoutée sous actor:{new_id}")

Zendaya ajoutée sous actor:1321


Delete the movie with title "The Room"

In [20]:
for key in r.scan_iter("movie:*"):
    if r.hget(key, "title") == "The Room":
        r.delete(key)
        print("The Room supprimé.")
        break